In [ ]:
# Config and imports

import pypsa
import logging
import atlite
import time
import warnings
import geopandas as gpd
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
from plotly import express as px
from plotly import graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.gridspec import GridSpec
import ipywidgets as widgets
from atlite.gis import shape_availability, ExclusionContainer
from prototype.lib.optimize import create_and_store_optimize
from prototype.lib.network import create_and_store_network
from prototype.lib.costs import create_and_store_costs
from prototype.lib.cutout import create_and_store_cutout
from prototype.lib.availability import create_and_store_availability
from prototype.lib.demand import create_and_store_demand
from prototype.lib.costs import read_assumptions

logging.basicConfig(level=logging.INFO)

In [ ]:
config = {
    "base-currency": "SEK",
    "exchange-rates": {
        "EUR": 11.68
    },
    "eur_to_sek": 11.68,
    "onwind_turbine": "2030_5MW_onshore.yaml",
    "offwind_turbine": "2030_20MW_offshore.yaml",
    "weather": 2023,
    "base-year": 2024,
    "target-year": 2035, 
    "costs": 2030,
    "discount-rate": 0.04,
    "demand": 2023,
    "load-target": 19,
    "network": {
        "nuclear": True,
        "h2": True,
        "offwind": True,
        "biogas": "Stor",
        "onwind-limit": "",
        "offwind-limit": ""
    },
    "geography": "14"
}

In [ ]:
base_year = config["base-year"]
target_year = config["target-year"]
discount_rate = config["discount-rate"]
base_currency = config["base-currency"]
exchange_rates = config["exchange-rates"]

assumptions = read_assumptions(f"data/assumptions.csv", base_year, target_year, base_currency, exchange_rates, discount_rate)

assumptions

In [ ]:
import pathlib

wind_turbine = pathlib.Path(f"./windturbine/{config['onwind_turbine']}")

wind_turbine


In [ ]:
# Setting variables based on config
DATA_PATH=CONFIG["scenario"]["data-path"]
WEATHER_START=CONFIG["scenario"]["weather_start"]
WEATHER_END=CONFIG["scenario"]["weather_end"]
LAN_CODE = CONFIG["scenario"]["geography_lan_code"]
KOM_CODE = CONFIG["scenario"]["geography_kom_code"]
YEAR = CONFIG["scenario"]["demand"]
TARGET = CONFIG["scenario"]["load-target"]

DATA_ROOT_PATH="data/result"
GEO_KEY = f"{LAN_CODE}-{WEATHER_START}-{WEATHER_END}"
GEO_DATA_PATH = f"{DATA_ROOT_PATH}/geo/{GEO_KEY}"
DEMAND_KEY = f"{YEAR}/{TARGET}"
DEMAND_DATA_PATH = f"{DATA_ROOT_PATH}/{DEMAND_KEY}"

## Wind turbines
WIND_TURBINE = CONFIG["onwind_turbine"]
WIND_TURBINE_OFFSHORE = CONFIG["offwind_turbine"]

## Cutout / selection
CUTOUT = atlite.Cutout(f"../{GEO_DATA_PATH}/cutout.nc")
SELECTION = gpd.read_file(f"../{GEO_DATA_PATH}/selection.shp")
EEZ = gpd.read_file(f"../{GEO_DATA_PATH}/eez.shp")
INDEX = pd.to_datetime(pd.read_csv(f"../{GEO_DATA_PATH}/time_index.csv")["0"])

# Cutout / availablity
AVAIL_SOLAR = xr.open_dataarray(f"../{GEO_DATA_PATH}/avail_solar.nc")
AVAIL_ONWIND = xr.open_dataarray(f"../{GEO_DATA_PATH}/avail_onwind.nc")
AVAIL_OFFWIND = xr.open_dataarray(f"../{GEO_DATA_PATH}/avail_offwind.nc")

AVAIL_CAPACITY_SOLAR = xr.open_dataarray(f"../{GEO_DATA_PATH}/avail_capacity_solar.nc")
AVAIL_CAPACITY_ONWIND = xr.open_dataarray(f"../{GEO_DATA_PATH}/avail_capacity_onwind.nc")
AVAIL_CAPACITY_OFFWIND = xr.open_dataarray(f"../{GEO_DATA_PATH}/avail_capacity_offwind.nc")

DEMAND = pd.read_csv(f"../{DEMAND_DATA_PATH}/demand.csv", index_col=0, parse_dates=[0])

NETWORK = pypsa.Network()
NETWORK.import_from_netcdf(f"../data/{DATA_PATH}/network.nc")

STATISTICS = pd.read_pickle(f"../data/{DATA_PATH}/statistics.pkl")

parameters = pd.read_csv("../data/assumptions.csv")
parameters.set_index(['technology', 'parameter'], inplace=True)


In [ ]:
import pandas as pd
import numpy as np

# Sample DataFrame
data = {
    'year': [2020, 2030, 2040, 2050],
    'value': [100, 200, 300, 400]
}
df = pd.DataFrame(data)

def average_value_in_range(df, start_year, end_year):
    # Create a full range of years
    full_range_years = np.arange(start_year, end_year + 1)
    
    # Interpolate values for each year in the full range
    df_interpolated = df.set_index('year').reindex(full_range_years).interpolate(method='linear').reset_index()
    print(df_interpolated)
    
    # Calculate the average value in the specified range
    average_value = df_interpolated['value'].mean()
    
    return average_value

# Define the range of years
start_year = 2024
end_year = 2035

# Calculate the average value in the specified range
avg_value = average_value_in_range(df, start_year, end_year)
print(f"The average value between {start_year} and {end_year} is {avg_value:.2f}")
print(avg_value)

In [ ]:
## OVERRIDING VARIABLES?
#CONFIG["demand-target"] = 0.05
# And then, to update any step, just uncomment and run
#create_and_store_costs(CONFIG)
#create_and_store_cutout(CONFIG)
#create_and_store_availability(CONFIG)
#create_and_store_demand(CONFIG)
#create_and_store_network(CONFIG)
#create_and_store_optimize(CONFIG)

STATISTICS


In [ ]:
# Exclude land use per solar/wind

# Source: https://www.uts.edu.au/oecm/renewable-resource-mapping
# (Classification mapping: https://collections.sentinel-hub.com/corine-land-cover/readme.html)
# With the exception of "41; Water bodies"

# Land-cover classes NOT included for solar areas:
EXCLUDED_SOLAR = [1,4,5,6,7,9,10,11,23,24,25,27,30,33,34,35,36,37,38,39,40,41,42,43,44] # 1.1.1,1.2.2,1.2.3,1.2.4,1.3.1,1.3.3,1.4.1,1.4.2,3.1.1,3.1.2,3.1.3,3.2.2,3.3.1,3.3.4,3.3.5,4.1.1,4.1.2,4.2.1,4.2.2,4.2.3,5.1.1,5.1.2,5.2.1,5.2.2,5.2.3
# Land-cover classes NOT included for wind energy areas:
EXCLUDED_WIND_NON_OCEAN = [1,2,3,4,5,6,7,9,10,11,15,16,17,23,24,25,27,30,31,33,34,35,36,37,38,39,40,41,42,43,44] # 1.1.1,1.1.2,1.2.1,1.2.2,1.2.3,1.2.4,1.3.1,1.3.3,1.4.1,1.4.2,2.2.1,2.2.2,2.2.3,3.1.1,3.1.2,3.1.3,3.2.2,3.3.1,3.3.2,3.3.4,3.3.5,4.1.1,4.1.2,4.2.1,4.2.2,4.2.3,5.1.1,5.1.2,5.2.1,5.2.2,5.2.3
INCLUDED_WIND_OCEAN = [44, 41] # 5.2.3, 5.1.2


CORINE = "../data/geo/corine.tif"
solar_excluder = ExclusionContainer()
onwind_excluder = ExclusionContainer()
offwind_excluder = ExclusionContainer()

solar_excluder.add_raster(CORINE, codes=EXCLUDED_SOLAR)
onwind_excluder.add_raster(CORINE, codes=EXCLUDED_WIND_NON_OCEAN)
offwind_excluder.add_raster(CORINE, codes=INCLUDED_WIND_OCEAN, invert=True)

solar_avail = CUTOUT.availabilitymatrix(SELECTION, solar_excluder)
onwind_avail = CUTOUT.availabilitymatrix(SELECTION, onwind_excluder)
offwind_avail = CUTOUT.availabilitymatrix(SELECTION, offwind_excluder)

In [ ]:
# Plot the dashboard
bus_colors = ["black", "yellow", "blue", "green", "lightblue", "red", "red", "red", "red"]
network = NETWORK

## Parameters
window_size=56 # Rolling average window size for charts

## Create solar availability map widget
solar_map = widgets.Output()
with solar_map:
    solar_excluder.plot_shape_availability(SELECTION, show_kwargs = { "cmap": "Reds" })
    plt.show()

## Create onwind availability map widget
onwind_map = widgets.Output()
with onwind_map:
    onwind_excluder.plot_shape_availability(SELECTION, show_kwargs = { "cmap": "Greens" })
    plt.show()

## Create offwind availability map widget
offwind_map = widgets.Output()
with offwind_map:
    offwind_excluder.plot_shape_availability(SELECTION, show_kwargs = { "cmap": "Blues" })
    plt.show()

## Create solar capacity factor line chart widget
solar_capacity_line_chart = widgets.Output()
with solar_capacity_line_chart:
    fig, ax = plt.subplots(figsize=(12, 4))
    AVAIL_CAPACITY_SOLAR.plot(ax=ax, color="Red", alpha=0.2)
    AVAIL_CAPACITY_SOLAR.rolling(time=window_size, center=True).mean().plot(ax=ax, color="Red")
    ax.set_ylabel("Capacity factor")
    ax.set_xlabel('')
    plt.show()

## Create onwind capacity factor line chart widget
onwind_capacity_line_chart = widgets.Output()
with onwind_capacity_line_chart:
    fig, ax = plt.subplots(figsize=(12, 4))
    AVAIL_CAPACITY_ONWIND.plot(ax=ax, color="Green", alpha=0.2)
    AVAIL_CAPACITY_ONWIND.rolling(time=window_size, center=True).mean().plot(ax=ax, color="Green")
    ax.set_ylabel("Capacity factor")
    ax.set_xlabel('')
    plt.show()

## Create offwind capacity factor line chart widget
offwind_capacity_line_chart = widgets.Output()
with offwind_capacity_line_chart:
    fig, ax = plt.subplots(figsize=(12, 4))
    AVAIL_CAPACITY_OFFWIND.plot(ax=ax, color="Blue", alpha=0.2)
    AVAIL_CAPACITY_OFFWIND.rolling(time=window_size, center=True).mean().plot(ax=ax, color="Blue")
    ax.set_ylabel("Capacity factor")
    ax.set_xlabel('')
    plt.show()

## Create target load line chart widget
load_line_chart = widgets.Output()
with load_line_chart:
    fig, ax = plt.subplots(figsize=(14, 7))
    DEMAND.plot(ax=ax, color='grey', alpha=0.2)
    DEMAND.rolling(window=window_size, center=True).mean().plot(ax=ax, color='blue', alpha=1)
    ax.legend(['Demand (3h average)', 'Demand (weekly average'])
    ax.set_ylabel("Load [MW]")
    ax.set_xlabel('')
    plt.show()


## Create network map widget
minx, miny, maxx, maxy = SELECTION.total_bounds
warnings.filterwarnings("ignore", category=UserWarning, module="cartopy.mpl.feature_artist")
network_map = widgets.Output()
with network_map:
    fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={'projection': ccrs.PlateCarree()})
    network.plot(ax=ax, boundaries = [minx-1, maxx+3, miny-1.5, maxy+4.5], color_geomap=True, bus_sizes=0.02, bus_colors=bus_colors)
    legend_labels = [plt.Line2D([0], [0], marker='o', color='w', label=bus, 
                            markerfacecolor=color, markersize=10) for bus, color in zip(network.buses.index, bus_colors)]
    plt.legend(handles=legend_labels, loc='upper left', bbox_to_anchor=(0.6, 0.7), ncol=1)
    plt.show()

## Format and create the parameters table widget
parameters_table = widgets.HTML(value=parameters.to_html())

## Format and create the system table widget widget
generator_column_names = {
    'capital_cost': 'Capital Cost',
    'marginal_cost': 'Marginal Cost',
    'lifetime': 'Lifetime',
    'p_nom_opt': 'Optimal Capacity'
}

store_column_names = {
    'capital_cost': 'Capital Cost',
    'marginal_cost': 'Marginal Cost',
    'lifetime': 'Lifetime',
    'e_nom_opt': 'Optimal Capacity'
}

generators = pd.concat([
    network.generators.loc[['Solar park', 'Onwind park', 'Offwind park']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']],
    network.links.loc[['Gas turbine']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']]
])

generators['capital_cost'] = generators['capital_cost'].apply(lambda x: f"{round(float(x)/1_000_000, 1)} MSEK/MW")
generators['marginal_cost'] = generators['marginal_cost'].apply(lambda x: f"{round(float(x), 1)} SEK/MWh")
generators['lifetime'] = generators['lifetime'].apply(lambda x: f"{int(round(float(x), 0))} years")
generators['p_nom_opt'] = generators['p_nom_opt'].apply(lambda x: f"{int(round(float(x), 0))} MW")
generators.rename(columns=generator_column_names, inplace=True)

stores = network.stores.loc[["H2 storage", "Battery storage"]][['capital_cost', 'marginal_cost', 'lifetime', 'e_nom_opt']]
stores.loc[:, 'capital_cost'] = stores['capital_cost'].apply(lambda x: f"{round(float(x)/1_000_000,1)} MSEK/MWh")
stores.loc[:, 'marginal_cost'] = stores['marginal_cost'].apply(lambda x: f"{round(float(x),1)} SEK/MWh")
stores.loc[:, 'lifetime'] = stores['lifetime'].apply(lambda x: f"{int(round(float(x), 0))} years")
stores.loc[:, 'e_nom_opt'] = stores['e_nom_opt'].apply(lambda x: f"{int(round(float(x), 0))} MWh")
stores.rename(columns=store_column_names, inplace=True)

system_table = widgets.HTML(value=pd.concat([generators, stores]).to_html())


## Format and create the stores table widget widget

## Create generators size pie chart widget
generator_pie_chart = widgets.Output()
with generator_pie_chart:
    network.generators['p_nom_opt'].plot(figsize=(4, 4), kind='pie', labels=None, legend=True, startangle=90, title='Generator sizes')
    plt.show()

## Create stores size pie chart widget
store_pie_chart = widgets.Output()
with store_pie_chart:
    network.stores.loc[["H2 storage", "Battery storage"]]['e_nom_opt'].plot(figsize=(4, 4), kind='pie', labels=None, legend=True, startangle=90, title='Energy store sizes')
    plt.axis('off')  # Turns off the axis
    plt.show()

## Create price line chart widget
price_line_chart = widgets.Output()
with price_line_chart:
    network.buses_t.marginal_price.plot(figsize=(24, 6), legend=True, title='Marginal price')
    plt.show()

## Create generators outpuExclusionContainert line chart widget
generator_line_chart = widgets.Output()
with generator_line_chart:
    fig, ax = plt.subplots(figsize=(16,6))
    network.generators_t.p.abs().plot(ax=ax, kind='area', stacked=True, legend=True, alpha=0.2, title='Power output')
    network.generators_t.p.abs().rolling(window=window_size, center=True).mean().plot(ax=ax, kind='area', stacked=True)
    plt.show()

## Create generators total output pie chart
generator_pie_chart = widgets.Output()
with generator_pie_chart:
    network.generators_t.p.sum().plot(figsize=(8, 6), kind='pie', labels=None, legend=True, title='Power output')
    plt.show()


## Create stores line chart widget
store_line_chart = widgets.Output()
with store_line_chart:
    network.stores_t.e[['H2 storage', 'Battery storage']].plot(figsize=(24, 6),legend=True, title='Energy stored')
    plt.show()


## Build the dashboard and show

sun_wind_map = widgets.HBox([solar_map, onwind_map, offwind_map])
sun_wind_cfs = widgets.VBox([solar_capacity_line_chart, onwind_capacity_line_chart, offwind_capacity_line_chart])

row_1 = widgets.HBox([widgets.VBox([load_line_chart, sun_wind_map]), sun_wind_cfs])

row_2 = widgets.HBox([widgets.VBox([network_map, system_table, widgets.HBox([generator_pie_chart, store_pie_chart])]), parameters_table])

row_3 = widgets.HBox([generator_line_chart, generator_pie_chart])

dashboard = widgets.VBox([row_1, row_2, row_3, store_line_chart, price_line_chart])

dashboard


In [ ]:
bus_colors = ["black", "yellow", "blue", "green", "lightblue", "red", "red", "red", "red", "black"]

minx, miny, maxx, maxy = SELECTION.total_bounds
warnings.filterwarnings("ignore", category=UserWarning, module="cartopy.mpl.feature_artist")
network_map = widgets.Output()
fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={'projection': ccrs.PlateCarree()})
NETWORK.plot(ax=ax, boundaries = [minx-1, maxx+3, miny-1.5, maxy+4.5], color_geomap=True, bus_sizes=0.02, bus_colors=bus_colors)
legend_labels = [plt.Line2D([0], [0], marker='o', color='w', label=bus, 
                        markerfacecolor=color, markersize=10) for bus, color in zip(network.buses.index, bus_colors)]
plt.legend(handles=legend_labels, loc='upper left', bbox_to_anchor=(0.6, 0.7), ncol=1)


In [ ]:
fix, ax1 = plt.subplots(figsize=(24, 6))
                        
NETWORK.buses_t.marginal_price[['Load bus']].plot(ax=ax1, legend=True, title='Marginal price')

ax2 = plt.twinx(ax1)
NETWORK.stores_t.e[['H2 storage']].plot(ax=ax2,legend=True, color='red', title='Energy stored')


In [ ]:
start_date = pd.to_datetime('2023-01-01')
end_date = pd.to_datetime('2023-12-31')

fix, ax1 = plt.subplots(figsize=(24, 6))
                        
NETWORK.buses_t.marginal_price[['Load bus']].loc[start_date:end_date].plot(ax=ax1, legend=True, title='Marginal price')

#ax2 = plt.twinx(ax1)
#DEMAND.plot(ax=ax2, color='grey', alpha=0.2)

#NETWORK.buses_t.marginal_price.loc[start_date:end_date].mean()


In [ ]:
total_cost = NETWORK.statistics.capex().sum()
total_energy = NETWORK.generators_t.p[['Solar park', 'Onwind park', 'Offwind park', 'Biogas market']].sum().sum()

total_cost/total_energy

In [ ]:
network.statistics.capex()

In [ ]:
NETWORK.generators_t.p.abs().rolling(window=56, center=True).mean().plot(figsize=(16,6), kind='area', stacked=True)

In [ ]:


NETWORK.generators_t.p.abs().plot(figsize=(16,6), kind='area', stacked=True)

In [ ]:
fix, ax1 = plt.subplots(figsize=(24, 6))                   
DEMAND.plot(ax=ax1, color='grey', alpha=0.2)

In [ ]:
column_names = {
    'annual_cost': 'Årskostnad',
    'lifetime': 'Livslängd',
    'p_nom_opt': 'Kapacitet',
    'generators': 'Kraftverk',
    'energy_produced': 'Producerad energi',
}

generator_index = NETWORK.generators.index.difference(['Backstop', 'Biogas input'])

generators = pd.concat([
    NETWORK.generators.loc[NETWORK.generators.index.isin(generator_index)][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt', 'p_nom_mod']],
    NETWORK.links.loc[['Combined Cycle Gas turbine', 'Simple Cycle Gas turbine']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt', 'p_nom_mod']]
])

generators['annual_cost'] = np.where(generators['p_nom_opt'] != 0, generators['capital_cost'] * generators['p_nom_opt'], 0)
generators['energy_produced'] = NETWORK.generators_t.p.loc[:, generator_index].sum() * 3
generators['generators'] = generators['p_nom_opt'] / generators['p_nom_mod']

other = NETWORK.links.loc[['H2 electrolysis', 'Battery charge']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']]
other['annual_cost'] = np.where(other['p_nom_opt'] != 0, other['capital_cost'] * other['p_nom_opt'], 0)

stores = NETWORK.stores.loc[["H2 storage", "Battery storage"]][['capital_cost', 'marginal_cost', 'lifetime', 'e_nom_opt']]
stores['annual_cost'] = np.where(stores['e_nom_opt'] != 0, stores['capital_cost'] * stores['e_nom_opt'], 0)
stores.rename(columns={'e_nom_opt': 'p_nom_opt'}, inplace=True)

biogas = NETWORK.generators.loc[['Biogas input']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']]
biogas['energy_produced'] = NETWORK.generators_t.p[['Biogas input']].sum() * 3 * float(parameters.loc['biogas', 'efficiency'].value)
biogas['annual_cost'] = np.where(biogas['p_nom_opt'] != 0, biogas['marginal_cost'] * biogas['energy_produced'] / float(parameters.loc['biogas', 'efficiency'].value), 0)
biogas[['lifetime', 'p_nom_opt', 'total_cost']] = 0

system = pd.concat([generators, other, stores, biogas]).drop(columns=['capital_cost', 'marginal_cost'])[['p_nom_opt', 'generators', 'lifetime', 'energy_produced', 'annual_cost']]

totals = pd.DataFrame(columns=system.columns, index=['Total'])
totals['energy_produced'] = system['energy_produced'].sum()
totals['annual_cost'] = system['annual_cost'].sum()

system = pd.concat([system, totals]).rename(columns=column_names)

years = lambda s: f'{s:,.0f} år'.replace(',', ' ')
million_cost = lambda s: f'{s/1_000_000:,.0f} MSEK'.replace(',', ' ')
energy = lambda s: f'{s/1_000:,.0f} GWh'.replace(',', ' ')
price = lambda s: f'{s:,.0f} SEK'.replace(',', ' ')
system.abs().style.format({'Livslängd': years, 'Producerad energi': energy, 'Pris': price, 'Årskostnad': million_cost, 'Totalkostnad': million_cost}, precision=0, na_rep='-')



In [ ]:
f"{system['Årskostnad']['Total'] / (NETWORK.loads_t.p['Desired load'].sum() * 3) / 1000 * 100:.1f} öre/kWh"

In [ ]:
170/23578

In [ ]:
start_date = pd.to_datetime('2023-08-01')
end_date = pd.to_datetime('2023-09-01')
fig1, ax1 = plt.subplots(figsize=(24, 6))
NETWORK.generators_t.p[['Solar park']].loc[start_date:end_date].plot(ax=ax1, kind='line', legend=True, alpha=0.2, title='Power output')
NETWORK.links_t.p0[['Solar battery link', 'Solar H2 link', 'Solar load link']].loc[start_date:end_date].plot(ax=ax1, kind='area', stacked=True, legend=True, alpha=0.2)

fig2, ax2 = plt.subplots(figsize=(24, 6))
NETWORK.generators_t.p[['Onwind park']].loc[start_date:end_date].plot(ax=ax2, kind='line', legend=True, alpha=0.2, title='Power output')
NETWORK.links_t.p0[['Onwind battery link', 'Onwind H2 link', 'Onwind load link']].loc[start_date:end_date].plot(ax=ax2, kind='area', stacked=True, legend=True, alpha=0.2)

fig3, ax3 = plt.subplots(figsize=(24, 6))
NETWORK.generators_t.p[['Offwind park']].loc[start_date:end_date].plot(ax=ax3, kind='line', legend=True, alpha=0.2, title='Power output')
NETWORK.links_t.p0[['Offwind battery link', 'Offwind H2 link', 'Offwind load link']].loc[start_date:end_date].plot(ax=ax3, kind='area', stacked=True, legend=True, alpha=0.2)

In [ ]:
solar_curtailment = NETWORK.generators_t.p_max_pu[['Solar park']] * NETWORK.generators.loc['Solar park']['p_nom_opt'] - NETWORK.generators_t.p[['Solar park']]
solar_curtailment['Curtailment'] = solar_curtailment['Solar park']
solar_curtailment.drop(columns=['Solar park'], inplace=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 6))

pd.concat([NETWORK.links_t.p0[['Solar battery link', 'Solar H2 link', 'Solar load link']], solar_curtailment]).resample('ME').sum().plot(ax=ax1, kind='bar', stacked=True, legend=True, alpha=0.2)

wind_curtailment = NETWORK.generators_t.p_max_pu[['Onwind park']] * NETWORK.generators.loc['Onwind park']['p_nom_opt'] - NETWORK.generators_t.p[['Onwind park']]
wind_curtailment['Curtailment'] = wind_curtailment['Onwind park']
wind_curtailment.drop(columns=['Onwind park'], inplace=True)

pd.concat([NETWORK.links_t.p0[['Onwind battery link', 'Onwind H2 link', 'Onwind load link']], wind_curtailment]).resample('ME').sum().plot(ax=ax2, kind='bar', stacked=True, legend=True, alpha=0.2)

In [ ]:
data = pd.concat([NETWORK.links_t.p0[['Solar battery link', 'Solar H2 link', 'Solar load link']], solar_curtailment]).resample('ME').sum()

# Prepare data for Plotly
data = data.reset_index()

# Create an interactive bar chart using Plotly
fig = go.Figure()

for column in data.columns[1:]:  # Skip the first column as it is the index
    fig.add_trace(go.Bar(
        x=data['snapshot'],  # Use the reset index for the x-axis
        y=data[column],
        name=column
    ))

# Update layout for better visualization
fig.update_layout(
    barmode='stack',
    xaxis_title='Date',
    yaxis_title='Energy (MWh)',
    title='Monthly Energy Flows and Curtailment',
    legend_title='Components',
    xaxis=dict(tickformat="%Y-%m"),
    hovermode='x unified'
)

# Show the interactive chart
fig.show()

In [ ]:
data = pd.concat([NETWORK.links_t.p0[['Solar battery link', 'Solar H2 link', 'Solar load link']], solar_curtailment]).resample('ME').sum()

# Prepare data for Plotly
data = data.reset_index()

# Create an interactive bar chart using Plotly
fig = go.Figure()

for column in data.columns[1:]:  # Skip the first column as it is the index
    fig.add_trace(go.Bar(
        x=data['snapshot'],  # Use the reset index for the x-axis
        y=data[column],
        name=column
    ))

# Update layout for better visualization
fig.update_layout(
    barmode='stack',
    xaxis_title='Date',
    yaxis_title='Energy (MWh)',
    title='Monthly Energy Flows and Curtailment',
    legend_title='Components',
    xaxis=dict(tickformat="%Y-%m"),
    hovermode='x unified'
)

# Show the interactive chart
fig.show()

curtailment = NETWORK.generators_t.p_max_pu[['Onwind park']] * NETWORK.generators.loc['Onwind park']['p_nom_opt'] - NETWORK.generators_t.p[['Onwind park']]
curtailment['Curtailment'] = curtailment['Onwind park']
curtailment.drop(columns=['Onwind park'], inplace=True)

fig1, ax1 = plt.subplots(figsize=(24, 6))
pd.concat([NETWORK.links_t.p0[['Onwind battery link', 'Onwind H2 link', 'Onwind load link']], curtailment]).resample('ME').sum().plot(ax=ax1, kind='bar', stacked=True, legend=True, alpha=0.2)

In [ ]:
NETWORK.loads_t.p[['Desired load']].sum()*3

In [ ]:
curtailment = NETWORK.generators_t.p_max_pu[['Solar park']] * NETWORK.generators.loc['Solar park']['p_nom_opt'] - NETWORK.generators_t.p[['Solar park']]

curtailment.plot(figsize=(24, 6), alpha=0.2)


In [ ]:
fig, ax = plt.subplots(figsize=(24,8))
network.generators_t.p[['Solar park']].abs().plot(ax=ax, kind='area', stacked=True, legend=True, alpha=0.2, title='Power output')
#network.generators_t.p.abs().rolling(window=window_size, center=True).mean().plot(ax=ax, kind='area', stacked=True)

In [ ]:
fig, ax = plt.subplots(figsize=(24,8))
network.stores_t.e[['Battery storage']].abs().plot(ax=ax, kind='area', stacked=True, legend=True, alpha=0.2, title='Power output')

In [ ]:
NETWORK.objective